# Hyperparameter Tuning for PyTorch With `spotpython` {#sec-hyperparameter-tuning-for-pytorch-14}

In this tutorial, we will show how `spotpython` can be integrated into the `PyTorch`
training workflow. It is based on the tutorial "Hyperparameter Tuning with Ray Tune" from the `PyTorch` documentation [@pyto23a], which is an extension of the tutorial "Training a Classifier" [@pyto23b] for training a CIFAR10 image classifier.

This document refers to the following software versions:

- ``python``: 3.10.10
- ``torch``: 2.0.1
- ``torchvision``: 0.15.0

In [ ]:
pip list | grep  "spot[RiverPython]"

`spotpython` can be installed via pip^[Alternatively, the source code can be downloaded from gitHub: [https://github.com/sequential-parameter-optimization/spotpython](https://github.com/sequential-parameter-optimization/spotpython).].

```{raw}
!pip install spotpython
```

* Uncomment the following lines if you want to for (re-)installation the latest version of `spotpython` from gitHub.

In [ ]:
# import sys
# !{sys.executable} -m pip install --upgrade build
# !{sys.executable} -m pip install --upgrade --force-reinstall spotpython

Results that refer to the `Ray Tune` package are taken from [https://PyTorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html](https://PyTorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)^[We were not able to install `Ray Tune` on our system. Therefore, we used the results from the `PyTorch` tutorial.].

## Setup {#sec-setup-14}

Before we consider the detailed experimental setup, we select the parameters that affect run time, initial design size and the device that is used.

In [ ]:
MAX_TIME = 30
INIT_SIZE = 10
DEVICE = "cpu" # "cuda:0"

In [ ]:
from spotpython.utils.device import getDevice
DEVICE = getDevice(DEVICE)
print(DEVICE)

In [ ]:
import os
import copy
import socket
import warnings
from datetime import datetime
from dateutil.tz import tzlocal
start_time = datetime.now(tzlocal())
HOSTNAME = socket.gethostname().split(".")[0]
experiment_name = '14-torch' + "_" + HOSTNAME + "_" + str(MAX_TIME) + "min_" + str(INIT_SIZE) + "init_" + str(start_time).split(".", 1)[0].replace(' ', '_')
experiment_name = experiment_name.replace(':', '-')
print(experiment_name)
if not os.path.exists('./figures'):
    os.makedirs('./figures')
warnings.filterwarnings("ignore")

## Initialization of the `fun_control` Dictionary {#sec-initialization-fun-control-14}

`spotpython` uses a Python dictionary for storing the information required for the hyperparameter tuning process. This dictionary is called `fun_control` and is initialized with the function `fun_control_init`. The function `fun_control_init` returns a skeleton  dictionary. The dictionary is filled with the required information for the hyperparameter tuning process. It stores the hyperparameter tuning settings, e.g., the deep learning network architecture that should be tuned, the classification (or regression) problem, and the data that is used for the tuning.
The dictionary is used as an input for the SPOT function.


In [ ]:
from spotpython.utils.init import fun_control_init
fun_control = fun_control_init(task="classification",
    tensorboard_path="runs/14_spot_ray_hpt_torch_cifar10",
    device=DEVICE,)

## PyTorch Data Loading {#sec-data-loading-14}

The data loading process is implemented in the same manner as described in the Section "Data loaders" in @pyto23a.
The data loaders are wrapped into the function `load_data_cifar10` which is identical to the function `load_data` in @pyto23a. A global data directory is used, which allows sharing the data directory between different trials.
The method `load_data_cifar10` is part of the `spotpython` package and can be imported from `spotpython.data.torchdata`.

In the following step, the test and train data are added to the dictionary `fun_control`.

In [ ]:
from spotpython.data.torchdata import load_data_cifar10
train, test = load_data_cifar10()
n_samples = len(train)
# add the dataset to the fun_control
fun_control.update({
    "train": train,
    "test": test,
    "n_samples": n_samples})

## The Model (Algorithm) to be Tuned {#sec-the-model-to-be-tuned-14}

### Specification of the Preprocessing Model {#sec-specification-of-preprocessing-model-14}

After the training and test data are specified and added to the `fun_control` dictionary, `spotpython` allows the specification of a data preprocessing pipeline, e.g., for the scaling of the data or for the one-hot encoding of categorical variables. The preprocessing model is called `prep_model` ("preparation" or pre-processing) and includes steps that are not subject to the hyperparameter tuning process. The preprocessing model is specified in the `fun_control` dictionary. The preprocessing model can be implemented as a `sklearn` pipeline. The following code shows a typical preprocessing pipeline:

```{raw}
categorical_columns = ["cities", "colors"]
one_hot_encoder = OneHotEncoder(handle_unknown="ignore",
                                    sparse_output=False)
prep_model = ColumnTransformer(
        transformers=[
             ("categorical", one_hot_encoder, categorical_columns),
         ],
         remainder=StandardScaler(),
     )
```

Because the Ray Tune (`ray[tune]`) hyperparameter tuning as described in @pyto23a does not use a preprocessing model, the preprocessing model is set to `None` here.

In [ ]:
prep_model = None
fun_control.update({"prep_model": prep_model})

### Select `algorithm` and `core_model_hyper_dict` {#sec-selection-of-the-algorithm}

The same neural network model as implemented in the section "Configurable neural network" of the `PyTorch` tutorial [@pyto23a] is used here.
We will show the implementation from @pyto23a in @sec-implementation-with-raytune first, before the extended implementation with `spotpython` is shown in @sec-implementation-with-spotpython-14.


#### Implementing a Configurable Neural Network With Ray Tune{#sec-implementation-with-raytune}

We used the same hyperparameters that are implemented as configurable in the `PyTorch` tutorial. We specify the layer sizes, namely `l1` and `l2`, of the fully connected layers:

```{raw}
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

```

The learning rate, i.e., `lr`,  of the optimizer is made configurable, too:

```{raw}
optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
```

#### Implementing a Configurable Neural Network With spotpython {#sec-implementation-with-spotpython-14}

`spotpython` implements a class which is similar to the class described in the `PyTorch` tutorial. The class is called `Net_CIFAR10` and is implemented in the file `netcifar10.py`.

```{raw}
from torch import nn
import torch.nn.functional as F
import spotpython.torch.netcore as netcore


class Net_CIFAR10(netcore.Net_Core):
    def __init__(self, l1, l2, lr_mult, batch_size, epochs, k_folds, patience,
    optimizer, sgd_momentum):
        super(Net_CIFAR10, self).__init__(
            lr_mult=lr_mult,
            batch_size=batch_size,
            epochs=epochs,
            k_folds=k_folds,
            patience=patience,
            optimizer=optimizer,
            sgd_momentum=sgd_momentum,
        )
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
```

### The `Net_Core` class {#sec-the-netcore-class-14}

`Net_CIFAR10` inherits from the class `Net_Core` which is implemented in the file `netcore.py`.  It implements the additional attributes that are common to all neural network models. The `Net_Core` class is implemented in the file `netcore.py`. It implements hyperparameters as attributes, that are not used by the `core_model`, e.g.:

* optimizer (`optimizer`),
* learning rate (`lr`),
* batch size (`batch_size`),
* epochs (`epochs`),
* k_folds (`k_folds`), and
* early stopping criterion "patience" (`patience`).

Users can add further attributes to the class. The class `Net_Core` is shown below.

```{raw}
from torch import nn


class Net_Core(nn.Module):
    def __init__(self, lr_mult, batch_size, epochs, k_folds, patience,
        optimizer, sgd_momentum):
        super(Net_Core, self).__init__()
        self.lr_mult = lr_mult
        self.batch_size = batch_size
        self.epochs = epochs
        self.k_folds = k_folds
        self.patience = patience
        self.optimizer = optimizer
        self.sgd_momentum = sgd_momentum
```


### Comparison of the Approach Described in the PyTorch Tutorial With spotpython {#sec-comparison}

Comparing the class `Net` from the `PyTorch` tutorial and the class `Net_CIFAR10` from `spotpython`, we see that the class `Net_CIFAR10` has additional attributes and does not inherit from `nn` directly. It adds an additional class, `Net_core`, that takes care of additional attributes that are common to all neural network models, e.g., the learning rate multiplier `lr_mult` or the batch size `batch_size`.

`spotpython`'s `core_model` implements an instance of the `Net_CIFAR10` class. In addition to the basic neural network model, the `core_model` can use these additional attributes.
`spotpython` provides methods for handling these additional attributes to guarantee 100% compatibility with the `PyTorch` classes. The method `add_core_model_to_fun_control` adds the hyperparameters and additional attributes to the `fun_control` dictionary. The method is shown below.

In [ ]:
from spotpython.torch.netcifar10 import Net_CIFAR10
from spotpython.data.torch_hyper_dict import TorchHyperDict
from spotpython.hyperparameters.values import add_core_model_to_fun_control
core_model = Net_CIFAR10
fun_control = add_core_model_to_fun_control(core_model=core_model,
                              fun_control=fun_control,
                              hyper_dict=TorchHyperDict,
                              filename=None)

## The Search Space: Hyperparameters {#sec-search-space-14}

In @sec-configuring-the-search-space-with-ray-tune, we first describe how to configure the search space with `ray[tune]` (as shown in @pyto23a) 
and then how to configure the search space with `spotpython` in @sec-configuring-the-search-space-with-spotpython.

### Configuring the Search Space With Ray Tune {#sec-configuring-the-search-space-with-ray-tune}

 Ray Tune's search space can be configured as follows [@pyto23a]:

```{raw}
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}
```
The ``tune.sample_from()`` function enables the user to define sample
methods to obtain hyperparameters. In this example, the ``l1`` and ``l2`` parameters
should be powers of 2 between 4 and 256, so either 4, 8, 16, 32, 64, 128, or 256.
The ``lr`` (learning rate) should be uniformly sampled between 0.0001 and 0.1. Lastly,
the batch size is a choice between 2, 4, 8, and 16.

At each trial, `ray[tune]` will randomly sample a combination of parameters from these
search spaces. It will then train a number of models in parallel and find the best
performing one among these. `ray[tune]` uses the ``ASHAScheduler`` which will terminate bad
performing trials early.

### Configuring the Search Space With spotpython {#sec-configuring-search-space-spotpython-14}

#### The `hyper_dict` Hyperparameters for the Selected Algorithm

`spotpython` uses `JSON` files for the specification of the hyperparameters.
Users can specify their individual `JSON` files, or they can use the `JSON` files provided by `spotpython`.
The `JSON` file for the `core_model` is called `torch_hyper_dict.json`.

In contrast to `ray[tune]`, `spotpython` can handle numerical, boolean, and categorical hyperparameters. They can be specified in the `JSON` file in a similar way as the numerical hyperparameters as shown below.
Each entry in the `JSON` file represents one hyperparameter with the following structure:
`type`, `default`, `transform`, `lower`, and `upper`.


```json
"factor_hyperparameter": {
    "levels": ["A", "B", "C"],
    "type": "factor",
    "default": "B",
    "transform": "None",
    "core_model_parameter_type": "str",
    "lower": 0,
    "upper": 2},
```

The corresponding entries for the `Net_CIFAR10` class are shown below.

```json
{"Net_CIFAR10":
    {
        "l1": {
            "type": "int",
            "default": 5,
            "transform": "transform_power_2_int",
            "lower": 2,
            "upper": 9},
        "l2": {
            "type": "int",
            "default": 5,
            "transform": "transform_power_2_int",
            "lower": 2,
            "upper": 9},
        "lr_mult": {
            "type": "float",
            "default": 1.0,
            "transform": "None",
            "lower": 0.1,
            "upper": 10},
        "batch_size": {
            "type": "int",
            "default": 4,
            "transform": "transform_power_2_int",
            "lower": 1,
            "upper": 4},
        "epochs": {
            "type": "int",
            "default": 3,
            "transform": "transform_power_2_int",
            "lower": 1,
            "upper": 4},
        "k_folds": {
            "type": "int",
            "default": 2,
            "transform": "None",
            "lower": 2,
            "upper": 3},
        "patience": {
            "type": "int",
            "default": 5,
            "transform": "None",
            "lower": 2,
            "upper": 10},
        "optimizer": {
            "levels": ["Adadelta",
                       "Adagrad",
                       "Adam",
                       "AdamW",
                       "SparseAdam",
                       "Adamax",
                       "ASGD",
                       "LBFGS",
                       "NAdam",
                       "RAdam",
                       "RMSprop",
                       "Rprop",
                       "SGD"],
            "type": "factor",
            "default": "SGD",
            "transform": "None",
            "class_name": "torch.optim",
            "core_model_parameter_type": "str",
            "lower": 0,
            "upper": 12},
        "sgd_momentum": {
            "type": "float",
            "default": 0.0,
            "transform": "None",
            "lower": 0.0,
            "upper": 1.0}
    }
}
```

### Modifying the Hyperparameters {#sec-modification-of-hyperparameters-14}

Ray tune [@pyto23a] does not provide a way to change the specified hyperparameters without re-compilation. However, `spotpython` provides functions for modifying the hyperparameters, their bounds and factors as well as for activating and de-activating hyperparameters without re-compilation of the Python source code. These functions are described in the following.

#### Modify `hyper_dict` Hyperparameters for the Selected Algorithm aka `core_model` {#sec-modification-of-default-values}

After specifying the model, the corresponding hyperparameters, their types and bounds are loaded from the `JSON` file `torch_hyper_dict.json`. After loading, the user can modify the hyperparameters, e.g., the bounds.
`spotpython` provides a simple rule for de-activating hyperparameters: If the lower and the upper bound are set to identical values, the hyperparameter is de-activated. This is useful for the hyperparameter tuning, because it allows to specify a hyperparameter in the `JSON` file, but to de-activate it in the `fun_control` dictionary. This is done in the next step.


#### Modify Hyperparameters of Type numeric and integer (boolean)

Since the hyperparameter `k_folds` is not used in the `PyTorch` tutorial, it is de-activated here by setting the lower and upper bound to the same value. Note, `k_folds` is of type "integer".

In [ ]:
from spotpython.hyperparameters.values import modify_hyper_parameter_bounds
fun_control = modify_hyper_parameter_bounds(fun_control, 
    "batch_size", bounds=[1, 5])
fun_control = modify_hyper_parameter_bounds(fun_control, 
    "k_folds", bounds=[0, 0])
fun_control = modify_hyper_parameter_bounds(fun_control, 
    "patience", bounds=[3, 3])

#### Modify Hyperparameter of Type factor

In a similar manner as for the numerical hyperparameters, the categorical hyperparameters can be modified.
New configurations can be chosen by adding or deleting levels. For example, the hyperparameter `optimizer` can be re-configured as follows:

In the following setting, two optimizers (`"SGD"` and `"Adam"`) will be compared during the `spotpython` hyperparameter tuning. The hyperparameter `optimizer` is active.

In [ ]:
from spotpython.hyperparameters.values import modify_hyper_parameter_levels
fun_control = modify_hyper_parameter_levels(fun_control,
     "optimizer", ["SGD", "Adam"])

The hyperparameter `optimizer` can be de-activated by choosing only one value (level), here: `"SGD"`.

In [ ]:
fun_control = modify_hyper_parameter_levels(fun_control, "optimizer", ["SGD"])

As discussed in @sec-optimizers, there are some issues with the LBFGS optimizer. Therefore, the usage of the LBFGS optimizer is not  deactivated in `spotpython` by default. However, the LBFGS optimizer can be activated by adding it to the list of optimizers.
`Rprop`  was removed, because it does perform very poorly (as some pre-tests have shown). However, it can also be activated by adding it to the list of optimizers.
Since `SparseAdam` does not support dense gradients, `Adam` was used instead.
Therefore, there are 10 default optimizers:

In [ ]:
fun_control = modify_hyper_parameter_levels(fun_control, "optimizer",
    ["Adadelta", "Adagrad", "Adam", "AdamW", "Adamax", "ASGD", 
    "NAdam", "RAdam", "RMSprop", "SGD"])

## Optimizers {#sec-optimizers-14}

@tbl-optimizers shows some of the optimizers available in `PyTorch`:

$a$ denotes (0.9,0.999), $b$ (0.5,1.2), and $c$ (1e-6, 50), respectively.
$R$ denotes `required, but unspecified`.
"m" denotes `momentum`, "w_d" `weight_decay`, "d" `dampening`, "n" `nesterov`, "r" `rho`,  "l_s" `learning rate for scaling delta`, "l_d" `lr_decay`, "b" `betas`, "l" `lambd`, "a" `alpha`, "m_d" for `momentum_decay`, "e" `etas`, and "s_s" for `step_sizes`. 

| Optimizer | lr  | m | w_d | d | n | r | l_s | l_d | b | l | a | m_d | e | s_s|
| :-------    |:-- |:- | :--   | :- | :- |:- | :-  | :-  | :- |:--  |:-  | :-  | :- | :- |
| Adadelta  | -   | -   |  0.    | -    | -    | 0.9 | 1.   | -        | -     | -     |-      |    -      |   -  | -    |
| Adagrad   |1e-2 | -   | 0.     | -    | -    | -   | -     | 0.       | -     | -     |-      | -         |   -  | -    |
| Adam      |1e-3 | -   | 0.     | -    | -    | -   | -     | -        |$a$| - |-      |   -       |   -  | -    |
| AdamW     |1e-3 | -   | 1e-2   | -    | -    | -   | -     | -        |$a$| - |  -    |       -   |   -  | -    |
| SparseAdam | 1e-3| -  | -      | -    | -    | -   | -     | -        |$a$|-  |-      | -         |   -  | -    |
| Adamax | 2e-3    | -  | 0.     | -    | -    | -   | -     | -        | $a$   |- |-      | -         |   -  | -    |
| ASGD   | 1e-2 |  .9  | 0.     | -    | F| -   | -     | -        | -        |1e-4| .75  |  -        |  -   | -    |
| LBFGS |  1.   | -     | -      | -    | -    | -   | -     | -        | -        | -  |-      | -         |  -   |   -  |
| NAdam | 2e-3  | -     | 0.     | -    | -    | -   | -     | -        |$a$| - |-      | 0         |   -  | -    |
| RAdam | 1e-3  | -     | 0.     | -    | -    | -   | -     | -        |$a$| - |-      | -         |   -  | -    |
| RMSprop | 1e-2| 0. |   0.      |  -   | -    | -   | -     | -        |$a$| - |-      | -         |   -  | -    |
| Rprop |  1e-2 | -  | -         | -    | -    | -   | -     | -        | -         | -| $b$| $c$ | -   | -   |
| SGD   | $R$ | 0.| 0.      | 0.   | F | -  | -     | -        | -         |- |-       | -         |   -  | -    |

: Optimizers available in PyTorch (selection).  The default values are shown in the table. {#tbl-optimizers}

`spotpython` implements an `optimization` handler that maps the optimizer names to the corresponding `PyTorch` optimizers.

:::{.callout-note}
## A note on LBFGS

We recommend deactivating `PyTorch`'s LBFGS optimizer, because it does not perform very well.
The  `PyTorch` documentation, see [https://pytorch.org/docs/stable/generated/torch.optim.LBFGS.html#torch.optim.LBFGS](https://pytorch.org/docs/stable/generated/torch.optim.LBFGS.html#torch.optim.LBFGS), states:

> This is a very memory intensive optimizer (it requires additional `param_bytes * (history_size + 1)` bytes). If it doesn’t fit in memory try reducing the history size, or use a different algorithm.

Furthermore, the LBFGS optimizer is not compatible with the `PyTorch` tutorial. The reason is that the LBFGS optimizer requires the `closure` function, which is not implemented in the `PyTorch` tutorial. Therefore, the `LBFGS` optimizer is recommended here.
Since there are ten optimizers in the portfolio, it is not recommended tuning the hyperparameters that effect one single optimizer only.
:::

:::{.callout-note}
## A note on the learning rate

`spotpython` provides a multiplier for the default learning rates, `lr_mult`, because optimizers use different learning rates. Using a multiplier for the learning rates might enable a simultaneous tuning of the learning rates for all optimizers. However, this is not recommended, because the learning rates are not comparable across optimizers. Therefore, we recommend fixing the learning rate for all optimizers if multiple optimizers are used. This can be done by setting the lower and upper bounds of the learning rate multiplier to the same value as shown below.

Thus, the learning rate, which affects the `SGD` optimizer, will be set to a fixed value. We choose the default value of `1e-3` for the learning rate, because it is used in other `PyTorch` examples (it is also the default value used by `spotpython` as defined in the `optimizer_handler()` method). We recommend tuning the learning rate later, when a reduced set of optimizers is fixed.
Here, we will demonstrate how to select in a screening phase the optimizers that should be used for the hyperparameter tuning.
:::


For the same reason, we will fix the `sgd_momentum` to `0.9`. 

In [ ]:
fun_control = modify_hyper_parameter_bounds(fun_control,
    "lr_mult", bounds=[1.0, 1.0])
fun_control = modify_hyper_parameter_bounds(fun_control,
    "sgd_momentum", bounds=[0.9, 0.9])

## Evaluation: Data Splitting {#sec-data-splitting-14}

The evaluation procedure requires the specification of the way how the data is split into a train and a test set and the loss function (and a metric).
As a default, `spotpython` provides a standard hold-out data split and cross validation.

### Hold-out Data Split

If a hold-out data split is used, the data will be partitioned into a training, a validation, and a test data set.
The split depends on the setting of the `eval` parameter. If `eval` is set to `train_hold_out`, one data set, usually the original training data set, is split into a new training and a validation data set. The training data set is used for training the model. The validation data set is used for the evaluation of the hyperparameter configuration and early stopping to prevent overfitting. In this case, the original test data set is not used.

::: {.callout-note}
`spotpython` returns the hyperparameters of the machine learning and deep learning models, e.g., number of layers, learning rate, or optimizer, but not the model weights. Therefore, after the SPOT run is finished, the corresponding model with the optimized architecture has to be trained again with the best hyperparameter configuration. The training is performed on the training data set. The test data set is used for the final evaluation of the model.

Summarizing, the following splits are performed in the hold-out setting:

1. Run `spotpython` with `eval` set to `train_hold_out` to determine the best hyperparameter configuration.
2. Train the model with the best hyperparameter configuration ("architecture")  on the training data set:  `train_tuned(model_spot, train, "model_spot.pt")`.
3. Test the model on the test data: `test_tuned(model_spot, test, "model_spot.pt")`

These steps will be exemplified in the following sections.
:::

In addition to this `hold-out` setting, `spotpython` provides another hold-out setting, where an explicit test data is specified by the user that will be used as the validation set. To choose this option, the `eval` parameter is set to `test_hold_out`. In this case, the training data set is used for the model training. Then, the explicitly defined test data set is used for the evaluation of the hyperparameter configuration (the validation).

### Cross-Validation

The cross validation setting is used by setting the `eval` parameter to `train_cv` or `test_cv`. In both  cases, the data set is split into $k$ folds. The model is trained on $k-1$ folds and evaluated on the remaining fold. This is repeated $k$ times, so that each fold is used exactly once for evaluation. The final evaluation is performed on the test data set. The cross validation setting is useful for small data sets, because it allows to use all data for training and evaluation. However, it is computationally expensive, because the model has to be trained $k$ times.

::: {.callout-note}
Combinations of the above settings are possible, e.g., cross validation can be used for training and hold-out for evaluation or *vice versa*. Also, cross validation can be used for training and testing. Because cross validation is not used in the `PyTorch` tutorial [@pyto23a], it is not considered further here.
:::

### Overview of the Evaluation Settings

#### Settings for the Hyperparameter Tuning

An overview of the training evaluations is shown in @tbl-eval-settings.
`"train_cv"` and `"test_cv"` use `sklearn.model_selection.KFold()` internally.
More details on the data splitting are provided in @sec-detailed-data-splitting (in the Appendix).

| `eval` | `train` | `test` | function | comment |
| --- | :-: | :-: | :----- | :----- |
| `"train_hold_out"` | $\checkmark$ |  |  `train_one_epoch()`, `validate_one_epoch()` for early stopping|  splits the `train` data set internally|
| `"test_hold_out"` | $\checkmark$ | $\checkmark$ | `train_one_epoch()`, `validate_one_epoch()` for early stopping  |use the `test data set` for `validate_one_epoch()` |
| `"train_cv"` | $\checkmark$ |              |  `evaluate_cv(net, train)`  | CV using the  `train` data set |
| `"test_cv"` |               | $\checkmark$ |  `evaluate_cv(net, test)` | CV using the  `test` data set . Identical to `"train_cv"`, uses only test data.|

: Overview of the evaluation settings. {#tbl-eval-settings}

#### Settings for the Final Evaluation of the Tuned Architecture

##### Training of the Tuned Architecture

`train_tuned(model, train)`: train the model with the best hyperparameter configuration (or simply the default) on the training data set. It splits the `train`data into new `train` and `validation` sets using  `create_train_val_data_loaders()`, which calls `torch.utils.data.random_split()` internally. Currently, 60% of the data is used for training and 40% for validation. The `train` data is used for training the model with `train_hold_out()`. The `validation` data is used for early stopping using `validate_fold_or_hold_out()` on the `validation` data set.

##### Testing of the Tuned Architecture

`test_tuned(model, test)`: test the model on the test data set. No data splitting is performed. The (trained) model is evaluated using the `validate_fold_or_hold_out()` function.
Note: During training, `"shuffle"` is set to `True`, whereas during testing, `"shuffle"` is set to `False`.

@sec-final-model-evaluation describes the final evaluation of the tuned architecture.

In [ ]:
fun_control.update({
    "eval": "train_hold_out",
    "path": "torch_model.pt",
    "shuffle": True})

## Evaluation: Loss Functions and Metrics {#sec-loss-functions-14}

The key `"loss_function"` specifies the loss function which is used during the optimization. There are several different loss functions under `PyTorch`'s `nn` package. For example, a simple loss is `MSELoss`, which computes the mean-squared error between the output and the target. In this tutorial we will use `CrossEntropyLoss`, because it is also used in the `PyTorch` tutorial.

In [ ]:
from torch.nn import CrossEntropyLoss
loss_function = CrossEntropyLoss()
fun_control.update({"loss_function": loss_function})

In addition to the loss functions, `spotpython` provides access to a large number of metrics.

* The key `"metric_sklearn"` is used for metrics that follow the `scikit-learn` conventions.
* The key `"river_metric"` is used for the river based evaluation [@mont20a] via `eval_oml_iter_progressive`, and 
* the key `"metric_torch"` is used for the metrics from `TorchMetrics`. 

`TorchMetrics` is a collection of more than 90 PyTorch metrics, see [https://torchmetrics.readthedocs.io/en/latest/](https://torchmetrics.readthedocs.io/en/latest/).
Because the `PyTorch` tutorial uses the accuracy as metric, we use the same metric here. Currently, accuracy is computed in the tutorial's example code. We will use `TorchMetrics` instead, because it offers more flexibilty, e.g., it can be used for regression and classification. Furthermore, `TorchMetrics` offers the following advantages:

    * A standardized interface to increase reproducibility
    * Reduces Boilerplate
    * Distributed-training compatible
    * Rigorously tested
    * Automatic accumulation over batches
    * Automatic synchronization between multiple devices

Therefore, we set 

In [ ]:
import torchmetrics
metric_torch = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(fun_control["device"])
fun_control.update({"metric_torch": metric_torch})

## Preparing the SPOT Call {#sec-prepare-spot-call-14}

The following code passes the information about the parameter ranges and bounds to `spot`.

In [ ]:
from spotpython.hyperparameters.values import (
    get_var_type,
    get_var_name,
    get_bound_values    
    )
var_type = get_var_type(fun_control)
var_name = get_var_name(fun_control)
fun_control.update({"var_type": var_type,
                    "var_name": var_name})

lower = get_bound_values(fun_control, "lower")
upper = get_bound_values(fun_control, "upper")

Now, the dictionary `fun_control` contains all information needed for the hyperparameter tuning. Before the hyperparameter tuning is started, it is recommended to take a look at the experimental design. The method `gen_design_table` generates a design table as follows:

In [ ]:
#| fig-cap: Experimental design for the hyperparameter tuning.
from spotpython.utils.eda import gen_design_table
print(gen_design_table(fun_control))

This allows to check if all information is available and if the information is correct.  @tbl-design shows the experimental design for the hyperparameter tuning. The table shows the hyperparameters, their types, default values, lower and upper bounds, and the transformation function. The transformation function is used to transform the hyperparameter values from the unit hypercube to the original domain. The transformation function is applied to the hyperparameter values before the evaluation of the objective function.  Hyperparameter transformations are shown in the column "transform", e.g., the `l1` default is `5`, which results in the value $2^5 = 32$ for the network, because the transformation ` transform_power_2_int` was selected in the `JSON` file. The default value of the `batch_size` is set to `4`, which results in a batch size of $2^4 = 16$. 

## The Objective Function `fun_torch` {#sec-the-objective-function-14}

The objective function `fun_torch` is selected next. It implements an interface from `PyTorch`'s training, validation, and  testing methods to `spotpython`.

In [ ]:
from spotpython.fun.hypertorch import HyperTorch
fun = HyperTorch().fun_torch

## Using Default Hyperparameters or Results from Previous Runs {#sec-default-hyperparameters}

We add the default setting to the initial design:

In [ ]:
from spotpython.hyperparameters.values import get_default_hyperparameters_as_array
hyper_dict=TorchHyperDict().load()
X_start = get_default_hyperparameters_as_array(fun_control, hyper_dict)

## Starting the Hyperparameter Tuning {#sec-call-the-hyperparameter-tuner-14}

The `spotpython` hyperparameter tuning is started by calling the `Spot` function. Here, we will run the tuner for approximately 30 minutes (`max_time`). Note: the initial design is always evaluated in the `spotpython` run. As a consequence, the run may take longer than specified by `max_time`, because the evaluation time of initial design (here: `init_size`, 10 points) is performed independently of `max_time`.
During the run, results from the training is shown. These results can be visualized with Tensorboard as will be shown in @sec-tensorboard-14.

In [ ]:
from spotpython.spot import spot
from math import inf
import numpy as np
spot_tuner = spot.Spot(fun=fun,
                   lower = lower,
                   upper = upper,
                   fun_evals = inf,
                   fun_repeats = 1,
                   max_time = MAX_TIME,
                   noise = False,
                   tolerance_x = np.sqrt(np.spacing(1)),
                   var_type = var_type,
                   var_name = var_name,
                   infill_criterion = "y",
                   n_points = 1,
                   seed=123,
                   log_level = 50,
                   show_models= False,
                   show_progress= True,
                   fun_control = fun_control,
                   design_control={"init_size": INIT_SIZE,
                                   "repeats": 1},
                   surrogate_control={"noise": True,
                                      "cod_type": "norm",
                                      "min_theta": -4,
                                      "max_theta": 3,
                                      "n_theta": len(var_name),
                                      "model_fun_evals": 10_000,
                                      "log_level": 50
                                      })
spot_tuner.run(X_start=X_start)

## Tensorboard {#sec-tensorboard-14}

The textual output shown in the console (or code cell) can be visualized with Tensorboard.

### Tensorboard: Start Tensorboard

Start TensorBoard through the command line to visualize data you logged. 
Specify the root log directory as used in `fun_control = fun_control_init(task="regression", tensorboard_path="runs/24_spot_torch_regression")` as the `tensorboard_path`. The argument logdir points to directory where TensorBoard will look to find event files that it can display. TensorBoard will recursively walk the directory structure rooted at logdir, looking for .*tfevents.* files.

```{raw}
tensorboard --logdir=runs
```

Go to the URL it provides or to [http://localhost:6006/](http://localhost:6006/).
The following figures show some screenshots of Tensorboard.

![Tensorboard](./figures/14-torch_p040025_10min_5init_2023-06-07_12-41-06_tensorboard_01.png){#fig-tensorboard_0}

![Tensorboard](./figures/14-torch_p040025_10min_5init_2023-06-07_12-41-06_tensorboard_02.png){#fig-tensorboard_hdparams}


### Saving the State of the Notebook {#sec-saving-the-state-of-the-notebook}

The state of the notebook can be saved and reloaded as follows:

In [ ]:
import pickle
SAVE = False
LOAD = False

if SAVE:
    result_file_name = "res_" + experiment_name + ".pkl"
    with open(result_file_name, 'wb') as f:
        pickle.dump(spot_tuner, f)

if LOAD:
    result_file_name = "add_the_name_of_the_result_file_here.pkl"
    with open(result_file_name, 'rb') as f:
        spot_tuner =  pickle.load(f)

## Results {#sec-results-14}

After the hyperparameter tuning run is finished, the progress of the hyperparameter tuning can be visualized. The following code generates the progress plot from @fig-progress.

In [ ]:
#| fig-cap: Progress plot. *Black* dots denote results from the initial design. *Red* dots  illustrate the improvement found by the surrogate model based optimization.
spot_tuner.plot_progress(log_y=False, 
    filename="./figures/" + experiment_name+"_progress.png")

@fig-progress shows a typical behaviour that can be observed in many hyperparameter studies [@bart21i]: the largest improvement is obtained during the evaluation of the initial design. The surrogate model based optimization-optimization with the surrogate refines the results. @fig-progress also illustrates one major difference between `ray[tune]` as used in @pyto23a and `spotpython`: the `ray[tune]` uses a random search and will generate results similar to the *black* dots, whereas `spotpython` uses a surrogate model based optimization and presents results represented by *red* dots in @fig-progress. The surrogate model based optimization is considered to be more efficient than a random search, because the surrogate model guides the search towards promising regions in the hyperparameter space.

In addition to the improved ("optimized") hyperparameter values, `spotpython` allows a statistical analysis, e.g., a sensitivity analysis, of the results. We can print the results of the hyperparameter tuning, see @tbl-results. The table shows the hyperparameters, their types, default values, lower and upper bounds, and the transformation function. The column "tuned" shows the tuned values. The column "importance" shows the importance of the hyperparameters. The column "stars" shows the importance of the hyperparameters in stars. The importance is computed by the SPOT software. 

In [ ]:
#| fig-cap: Results of the hyperparameter tuning.
from spotpython.utils.eda import gen_design_table
print(gen_design_table(fun_control=fun_control, spot=spot_tuner))

To visualize the most important hyperparameters, `spotpython` provides the function `plot_importance`. The following code generates the importance plot from @fig-importance.

In [ ]:
#| fig-cap: 'Variable importance plot, threshold 0.025.'
spot_tuner.plot_importance(threshold=0.025,
    filename="./figures/" + experiment_name+"_importance.png")

### Get the Tuned Architecture (SPOT Results) {#sec-get-spot-results-14}

The architecture of the `spotpython` model can be obtained as follows.
First, the numerical representation of the hyperparameters are obtained, i.e., the numpy array `X` is generated. This array is then used to generate the model `model_spot` by the function `get_one_core_model_from_X`. The model `model_spot` has the following architecture:

In [ ]:
from spotpython.hyperparameters.values import get_one_core_model_from_X
X = spot_tuner.to_all_dim(spot_tuner.min_X.reshape(1,-1))
model_spot = get_one_core_model_from_X(X, fun_control)
model_spot

### Get Default Hyperparameters

In a similar manner as in @sec-get-spot-results, the default hyperparameters can be obtained.

In [ ]:
# fun_control was modified, we generate a new one with the original 
# default hyperparameters
from spotpython.hyperparameters.values import get_one_core_model_from_X
fc = fun_control
fc.update({"core_model_hyper_dict":
    hyper_dict[fun_control["core_model"].__name__]})
model_default = get_one_core_model_from_X(X_start, fun_control=fc)
model_default

### Evaluation of the Default Architecture 

The method `train_tuned` takes a model architecture without trained weights and trains this model with the train data.
The train data is split into train and validation data. The validation data is used for early stopping. The trained model weights are saved as a dictionary.

This evaluation is similar to the final evaluation in @pyto23a. 


In [ ]:
from spotpython.torch.traintest import (
    train_tuned,
    test_tuned,
    )
train_tuned(net=model_default, train_dataset=train, shuffle=True,
        loss_function=fun_control["loss_function"],
        metric=fun_control["metric_torch"],
        device = fun_control["device"], show_batch_interval=1_000_000,
        path=None,
        task=fun_control["task"],)

test_tuned(net=model_default, test_dataset=test, 
        loss_function=fun_control["loss_function"],
        metric=fun_control["metric_torch"],
        shuffle=False, 
        device = fun_control["device"],
        task=fun_control["task"],)        

### Evaluation of the Tuned Architecture 

The following code trains the model `model_spot`.

If `path` is set to a filename, e.g., `path = "model_spot_trained.pt"`, the weights of the trained model will be saved to this file.

If `path` is set to a filename, e.g., `path = "model_spot_trained.pt"`, the weights of the trained model will be loaded from this file.

In [ ]:
train_tuned(net=model_spot, train_dataset=train,
        loss_function=fun_control["loss_function"],
        metric=fun_control["metric_torch"],
        shuffle=True,
        device = fun_control["device"],
        path=None,
        task=fun_control["task"],)
test_tuned(net=model_spot, test_dataset=test,
            shuffle=False,
            loss_function=fun_control["loss_function"],
            metric=fun_control["metric_torch"],
            device = fun_control["device"],
            task=fun_control["task"],)

### Detailed Hyperparameter Plots

The contour plots in this section visualize the interactions of the three most important hyperparameters. Since some of these hyperparameters take fatorial or integer values, sometimes step-like fitness landcapes (or response surfaces) are generated.
SPOT draws the interactions of the main hyperparameters by default. It is also possible to visualize all interactions. 

In [ ]:
#| fig-cap: Contour plots.
filename = "./figures/" + experiment_name
spot_tuner.plot_important_hyperparameter_contour(filename=filename)

The figures (@fig-contour) show the contour plots of the loss as a function of the hyperparameters.
These plots are very helpful for benchmark studies and for understanding neural networks.
 `spotpython` provides additional tools for a visual inspection of the results and give valuable insights into the hyperparameter tuning process.
 This is especially useful for model explainability, transparency, and trustworthiness.
In addition to the contour plots, @fig-parallel shows the parallel plot of the hyperparameters.

In [ ]:
#| fig-cap: Parallel coordinates plots
spot_tuner.parallel_plot()

## Summary and Outlook {#sec-summary}

This tutorial presents the hyperparameter tuning open source software `spotpython` for `PyTorch`.
To show its basic features, a comparison with the "official" `PyTorch` hyperparameter tuning tutorial [@pyto23a] is presented.
Some of the advantages of `spotpython` are:

* Numerical and categorical hyperparameters.
* Powerful surrogate models.
* Flexible approach and easy to use.
* Simple JSON files for the specification of the hyperparameters.
* Extension of default and user specified network classes.
* Noise handling techniques.
* Interaction with `tensorboard`.

Currently, only rudimentary parallel and distributed neural network training is possible,
but these capabilities will be extended in the future. The next version of `spotpython` will also include a more detailed documentation and more examples.

::: {.callout-important}
Important: This tutorial does not present a complete benchmarking study [@bart20gArxiv]. The results are only preliminary and highly dependent on the local configuration (hard- and software). Our goal is to provide a first impression of the performance of the hyperparameter tuning package `spotpython`. To demonstrate its capabilities, a quick comparison with `ray[tune]` was performed. `ray[tune]` was chosen, because it is presented as "an industry standard tool for distributed hyperparameter tuning."  The results should be interpreted with care.
:::


## Appendix {#sec-appendix}

### Sample Output From Ray Tune's Run

The output from `ray[tune]` could look like this [@pyto23b]:

```{raw}
Number of trials: 10 (10 TERMINATED)
------+------+-------------+--------------+---------+------------+--------------------+
|   l1 |   l2 |          lr |   batch_size |    loss |   accuracy | training_iteration |
+------+------+-------------+--------------+---------+------------+--------------------|
|   64 |    4 | 0.00011629  |            2 | 1.87273 |     0.244  |                  2 |
|   32 |   64 | 0.000339763 |            8 | 1.23603 |     0.567  |                  8 |
|    8 |   16 | 0.00276249  |           16 | 1.1815  |     0.5836 |                 10 |
|    4 |   64 | 0.000648721 |            4 | 1.31131 |     0.5224 |                  8 |
|   32 |   16 | 0.000340753 |            8 | 1.26454 |     0.5444 |                  8 |
|    8 |    4 | 0.000699775 |            8 | 1.99594 |     0.1983 |                  2 |
|  256 |    8 | 0.0839654   |           16 | 2.3119  |     0.0993 |                  1 |
|   16 |  128 | 0.0758154   |           16 | 2.33575 |     0.1327 |                  1 |
|   16 |    8 | 0.0763312   |           16 | 2.31129 |     0.1042 |                  4 |
|  128 |   16 | 0.000124903 |            4 | 2.26917 |     0.1945 |                  1 |
+-----+------+------+-------------+--------------+---------+------------+--------------------+
Best trial config: {'l1': 8, 'l2': 16, 'lr': 0.00276249, 'batch_size': 16, 'data_dir': '...'}
Best trial final validation loss: 1.181501
Best trial final validation accuracy: 0.5836
Best trial test set accuracy: 0.5806
```
